# Interacting with CLIP

This is a self-contained notebook that shows how to download and run CLIP models, calculate the similarity between arbitrary image and text inputs, and perform zero-shot image classifications.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
root_path = 'gdrive/My Drive/iiitd_research'

In [ ]:
import os
print(os.listdir())
os.chdir('gdrive/My Drive/iiitd_research')
#!unzip 'constraint22_dataset_covid19.zip'
# print(os.getcwd())
# print(os.listdir('./content/covid_meme_data'))


#os.chdir('/content/view?usp=sharing')

['.config', 'gdrive', 'sample_data']


In [ ]:
os.listdir()

In [ ]:
#!unzip 'constraint22_dataset_covid19.zip'

In [ ]:
# !unzip 'constraint22_dataset_uspolitics.zip'



In [ ]:
os.listdir('annotations')

['val.jsonl', 'train.jsonl', 'train2.jsonl', 'val2.jsonl']

Extracting data for training

In [ ]:

import json
with open('./annotations/val2.jsonl') as f:
    data = [json.loads(line) for line in f]
print(data[4])

{'OCR': 'MARIA BUTINA USED SEX\nTO INFILTRATE THE NRA\n& THE REPUBLICAN PARTY\nSO THERE MAY BE SEX TAPES\nOF MITCH MCCONNELL!\n', 'image': 'memes_4544.png', 'hero': [], 'villain': ['mitch mcconnell'], 'victim': [], 'other': ['national rifle association (nra)', 'nra', 'maria butina', 'republican party']}


In [ ]:
type(data)

list

In [ ]:
data_new = []
k=0
for i in data :

  i['entities']=i['hero']+i['villain']+i['victim']+i['other']
  #if len(i["OCR"])<=1500:
  for en in i['entities']:
      new_dict={}

      new_dict['image'] = i['image']
      new_dict['OCR'] = i['OCR']
      new_dict['entity'] = en
      if en in i['hero']:
        new_dict['label'] =0 #'hero'
      elif en in i['villain'] :
        new_dict['label'] =1 #'villain'
      elif en in i['victim'] :
        new_dict['label'] =2 #'victim'
      else:
        new_dict['label'] =3 #'other'

      data_new.append(new_dict)
  k=k+1

In [ ]:
print(len(data_new))
print(k)
print(data_new[:5])

823
300
[{'image': 'covid_memes_2457.png', 'OCR': "Herman Caino\nOTHEHermancain\nMasks will not be mandatory for the event,\nwhich will be attended by President\nTrump. PEOPLE ARE FED UP!\nHERE'S SOME COVID FOR YOU\nAND VOU, AND YOU, AND YOU,\n", 'entity': 'herman cain', 'label': 1}, {'image': 'covid_memes_2457.png', 'OCR': "Herman Caino\nOTHEHermancain\nMasks will not be mandatory for the event,\nwhich will be attended by President\nTrump. PEOPLE ARE FED UP!\nHERE'S SOME COVID FOR YOU\nAND VOU, AND YOU, AND YOU,\n", 'entity': 'donald trump', 'label': 3}, {'image': 'covid_memes_2457.png', 'OCR': "Herman Caino\nOTHEHermancain\nMasks will not be mandatory for the event,\nwhich will be attended by President\nTrump. PEOPLE ARE FED UP!\nHERE'S SOME COVID FOR YOU\nAND VOU, AND YOU, AND YOU,\n", 'entity': 'covid', 'label': 3}, {'image': 'covid_memes_2457.png', 'OCR': "Herman Caino\nOTHEHermancain\nMasks will not be mandatory for the event,\nwhich will be attended by President\nTrump. PEOPLE A

In [ ]:
max=0

avg=0
for i in data_new:
  avg+=len(i['OCR'])
  if len(i['OCR'])>max:
    max=len(i['OCR'])


print(max,avg/10280)

1069 146.6580739299611


In [ ]:
print(data[4])

{'OCR': 'Occupy Danocrats\na at 12\nDREAKNG: Barack Cbaria andoruas Jos lidan for president wtt\nRALLIANT speech\nFolon Cetuny Desnouta ler reare.\nBREAKING:\nBarack Obama endorses\nJoe Biden for president\nJoe Biden for Presideht of the United States.\nK Corers a sras. 2a Vies\nO comment\nA Shere\n', 'image': 'covid_memes_622.png', 'hero': ['joe biden'], 'villain': [], 'victim': [], 'other': ['barack obama'], 'entities': ['joe biden', 'barack obama']}


In [ ]:
#os.listdir('images')

# Preparation for Colab

Make sure you're running a GPU runtime; if not, select "GPU" as the hardware accelerator in Runtime > Change Runtime Type in the menu. The next cells will install the `clip` package and its dependencies, and check if PyTorch 1.7.1 or later is installed.

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 946 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-yhoanb11
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-yhoanb11
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=7763ebce1e426f09fdf5f2764e56be78fa43b35bd7d0ea709448c6529e66a50c
  Stored in directory: /tmp/pip-ephem-wheel-cache-qwng5phz/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [ ]:
import numpy as np
import torch

print("Torch version:", torch.__version__)
torch.__version__#.split(".") #>= ["1", "7", "1"], "PyTorch 1.7.1 or later is required"

Torch version: 1.11.0+cu113


'1.11.0+cu113'

# Loading the model

`clip.available_models()` will list the names of available CLIP models.

In [ ]:
# import torch, gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [ ]:
model, preprocess = clip.load("ViT-L/14")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

100%|████████████████████████████████████████| 890M/890M [00:07<00:00, 126MiB/s]


Model parameters: 427,616,513
Input resolution: 224
Context length: 77
Vocab size: 49408


# Image Preprocessing

We resize the input images and center-crop them to conform with the image resolution that the model expects. Before doing so, we will normalize the pixel intensity using the dataset mean and standard deviation.

The second return value from `clip.load()` contains a torchvision `Transform` that performs this preprocessing.



In [ ]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x7f90ee7e65f0>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

# Text Preprocessing

We use a case-insensitive tokenizer, which can be invoked using `clip.tokenize()`. By default, the outputs are padded to become 77 tokens long, which is what the CLIP models expects.

In [ ]:
clip.tokenize("Hello World!")

tensor([[49406,  3306,  1002,   256, 49407,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]], dtype=torch.int32)

# Setting up input images and texts

We are going to feed 8 example images and their textual descriptions to the model, and compare the similarity between the corresponding features.

The tokenizer is case-insensitive, and we can freely give any suitable textual descriptions.

In [ ]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# images in skimage to use and their textual descriptions
# descriptions = {
#     "page": "a page of text about segmentation",
#     "chelsea": "a facial photo of a tabby cat",
#     "astronaut": "a portrait of an astronaut with the American flag",
#     "rocket": "a rocket standing on a launchpad",
#     "motorcycle_right": "a red motorcycle standing in a garage",
#     "camera": "a person looking at a camera on a tripod",
#     "horse": "a black-and-white silhouette of a horse",
#     "coffee": "a cup of coffee on a saucer"
# }
# descriptions = {}
# for image_desc in data:
#   if len(image_desc["OCR"])<77:
#     descriptions[image_desc["image"]] = image_desc["OCR"]

# print(descriptions)

In [ ]:
#(os.listdir('images'))

In [ ]:
original_images = []
images = []
ocr = []
entity=[]
labels=[]

n=0
temp=''
for i in data_new:
    # n+=1
    # if n==1004:
    #   break

    if i['image'] not in os.listdir('images'): #or i['image'][0]=='c':
        continue


    # if i['image']!=temp:
    #   image = Image.open(os.path.join('images/', i['image'])).convert("RGB")
    #   images.append(preprocess(image))
    #   temp=i['image']
    # else:
    #   images.append(images[-1])


    # ocr.append(i['OCR'])
    # entity.append(i['entity'])
    labels.append(i['label'])







In [ ]:
print(len(labels))
print(labels[9])

1246
3


In [ ]:
import pickle
with open("uspoliticsvallabels.pickle", 'wb') as f:
    pickle.dump(labels, f)


In [ ]:
len(images)

1246

In [ ]:
print(len(ocr))
print(len(entity))

1246
1246


In [ ]:
# import pickle
# with open("trainimagesallrn50x64.pickle", 'wb') as f:
#     pickle.dump(images, f)



In [ ]:
# with open("trainimagesallrn50x64.pickle", 'rb') as f:
#     images = pickle.load(f)

In [ ]:
# image_input=torch.empty(7234,3,224,224)

# def generator(k=50):
#   n=0
#   for i in data_new:
#     if i['image'] not in os.listdir('images'):
#         continue


#     image = Image.open(os.path.join('images/', i['image'])).convert("RGB")


#        #image_input = torch.tensor(np.stack(images)).cuda()
#      #original_images.append(image)
#     images.append(preprocess(image))
#     # n+=1
    # if n%k==0 and n!=0:
    #   temp = np.stack(images[n-k:n])
    #   yield torch.tensor(temp)#.cuda()
    #   #image_input = torch.cat((image_input.cuda(),))
    # elif n>7200:
    #   #image_input = torch.cat((image_input.cuda(),torch.tensor(np.stack(images[n:]).cuda()))).cuda()
    #   yield torch.tensor(np.stack(images[n:])).cuda()
    #   break

 # image_input=torch.tensor(image_input).cuda()

In [ ]:
# l=0
# for x in generator():
#   torch.cuda.empty_cache()
#   l+=1
#   print(l)
#   image_input = torch.cat((image_input,x)).cuda()


In [ ]:
len(images)

10280

## Building features

We normalize the images, tokenize each text input, and run the forward pass of the model to get the image and text features.

In [ ]:
#images[0].shape

In [ ]:

#image_input = torch.tensor(np.stack(images)).cuda()

# ocr_tokens = clip.tokenize([ o for o in ocr],truncate=True).cuda()
# entity_tokens = clip.tokenize([ e for e in entity]).cuda()

In [ ]:
k=0
image_features = []#torch.Tensor(3000,640)
for i in range(124):

  image_input = torch.tensor(np.stack(images[k:k+10])).cuda()
  k+=10
  with torch.no_grad():
    image_features=image_features + model.encode_image(image_input).float().tolist()
    #image_features = torch.cat(image_features, model.encode_image(image_input).float())

image_input = torch.tensor(np.stack(images[k:])).cuda()
with torch.no_grad():
    image_features=image_features + model.encode_image(image_input).float().tolist()

In [ ]:
# from torch.utils.data import DataLoader
# training_data = (image_input,ocr_tokens,entity_tokens)
# train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)

In [ ]:
# image_features =[]
# ocr_features=[]
# entity_features=[]

# epochs=10
# i=0
# while (i<len(image_input)):

#         if (i+100)<len(image_input):
#           image_input,ocr_tokens,entity_tokens =image_input[i:i+100].cuda(),ocr_tokens[i:i+100].cuda(),entity_tokens[i:i+100].cuda()

#         if (i+100)>=len(image_input):
#           image_input,ocr_tokens,entity_tokens =image_input[i:].cuda(),ocr_tokens[i:].cuda(),entity_tokens[i:].cuda()
#         i=i+100

#           #images, labels = images.cuda(), labels.cuda()
#         with torch.no_grad():

#           image_features.append(model.encode_image(image_input).float())
#           ocr_features.append(model.encode_text(ocr_tokens).float())
#           entity_features.append(model.encode_text(entity_tokens).float())


In [ ]:
k=0
# ocr_tokens = clip.tokenize([ o for o in ocr],truncate=True).cuda()
# entity_tokens = clip.tokenize([ e for e in entity]).cuda()
ocr_features = []
entity_features = []
for i in range(124):

  o = clip.tokenize(ocr[k:k+10],truncate=True).cuda()
  e = clip.tokenize(entity[k:k+10]).cuda()

  k+=10
  with torch.no_grad():
    ocr_features=ocr_features + model.encode_text(o).float().tolist()
    entity_features=entity_features + model.encode_text(e).float().tolist()
    #image_features = torch.cat(image_features, model.encode_image(image_input).float())

o = clip.tokenize(ocr[k:],truncate=True).cuda()
e = clip.tokenize(entity[k:]).cuda()
with torch.no_grad():
    ocr_features=ocr_features + model.encode_text(o).float().tolist()
    entity_features=entity_features + model.encode_text(e).float().tolist()

In [ ]:
print(len(ocr_features))
print(len(entity_features))

1246
1246


In [ ]:
# ocr_features = []
# entity_features = []

# with torch.no_grad():
#     #image_features= model.encode_image(image_input).float()
#     ocr_features = model.encode_text(ocr_tokens).float()
#     entity_features = model.encode_text(entity_tokens).float()

**Embeddings**

In [ ]:
image_features= torch.Tensor(image_features).cuda()
ocr_features= torch.Tensor(ocr_features).cuda()
entity_features= torch.Tensor(entity_features).cuda()

In [ ]:
print(type(image_features))
print(type(ocr_features))
type(entity_features)
type(image_features)

<class 'torch.Tensor'>
<class 'torch.Tensor'>


torch.Tensor

In [ ]:
print(len(image_features))
print(len(ocr_features))
print(len(entity_features))

print(len(labels))


1246
1246
1246
1246


In [ ]:
print((image_features[0]).shape)
print(ocr_features[0].shape)
print(entity_features[0].shape)

torch.Size([768])
torch.Size([768])
torch.Size([768])


In [ ]:
print(ocr_features[0])
print(entity_features[0])
print(image_features[0])

In [ ]:
print(ocr_features.is_cuda)
print(entity_features.is_cuda)
print(image_features.is_cuda)
#image_features= image_features.cuda()
print(image_features.is_cuda)

True
True
True
True


concatenating

In [ ]:
input_tensor = torch.cat((image_features, ocr_features, entity_features), 1)


In [ ]:
input_tensor.shape

torch.Size([1246, 2304])

Saving training embeddings

In [ ]:
import pickle
with open("uspoliticsViT-L14valembed.pickle", 'wb') as f:
    pickle.dump(input_tensor, f)



In [ ]:
# import pickle
# with open("finaltrainembed.pickle", 'rb') as f:
#     x = pickle.load(f)

In [ ]:
x.shape

NameError: ignored

In [ ]:
print(len(input_tensor))
print(input_tensor.shape)
print(input_tensor)

# **Trying clip**

In [ ]:
# for i in test_tensor[:2]:

#   print(i)

In [ ]:
# yfinal = []
# for i in test_tensor:
#   y = [sum(value) for value in zip(i[:512], i[512:512*2],i[512*2:])]
#   y = [x/3 for x in y]
#   yfinal.append(torch.Tensor(y).cuda())




In [ ]:
# yfinal =torch.Tensor(yfinal)

In [ ]:
# input_tensor.type

In [ ]:
# for i in range(len(yfinal)):
#    yfinal[i] = yfinal[i].to(device)

# print(yfinal[7])

In [ ]:
# image_input = preprocess(image).unsqueeze(0).to(device)
# qw =['hero','villian','victim','other']
# diffpred = []
# txt = torch.cat([clip.tokenize(f"a photo of a {c}") for c in qw]).to(device)

# # Calculate features
# with torch.no_grad():

#     txtfea = model.encode_text(txt).float()
# for i in range(1290):
#       im =yfinal[i]
#       # Pick the top 5 most similar labels for the image
#       im /= im.norm(dim=-1, keepdim=True)
#       txtfea /= txtfea.norm(dim=-1, keepdim=True)
#       similarity = (100.0 * im @ txtfea.T).softmax(dim=-1)
#       values, indices = similarity.topk(1)
#       index =torch.Tensor.cpu(indices).numpy()
#       diffpred.append(int(index))
# # Print the result
# # print("\nTop predictions:\n")
# # for value, index in zip(values, indices):
# #     print(f"{qw[index]}: {100 * value.item():.2f}%")



In [ ]:
# print(len(diffpred))

In [ ]:
# print(len(testlabels))

In [ ]:
# f1_score(testlabels, diffpred, average='macro')

In [ ]:
# import pickle
# with open("bert1000trainembeddings.pickle", 'rb') as f:
#     bert_train_embeddings = pickle.load(f)

In [ ]:
# bert_train_embeddings = torch.Tensor(bert_train_embeddings).cuda()

In [ ]:
# import pickle
# with open("bert1000entitytrainembeddings.pickle", 'rb') as f:
#     bert_entity_train_embeddings = pickle.load(f)

# bert_entity_train_embeddings = torch.Tensor(bert_entity_train_embeddings).cuda()

In [ ]:
# input_tensor[:1000].shape

In [ ]:
# bert_train_embeddings.shape

In [ ]:
# bert_entity_train_embeddings.shape

In [ ]:
# input_tensor = torch.cat((input_tensor[:1000],bert_train_embeddings,bert_entity_train_embeddings), 1)

In [ ]:
# os.listdir()
# !mkdir 'test_data'
# !unzip 'constraint22_dataset_unseen_test.zip' -d 'test_data'

# Preparing test embeddings


In [ ]:
os.listdir()

['data',
 'constraint22_dataset_covid19.zip',
 'annotations',
 'images',
 'constraint22_dataset_uspolitics.zip',
 'constraint22_dataset_unseen_test.zip',
 'test_data',
 'constraint22_dataset_unseen_test_gold_labels.jsonl',
 'trainimagesallrn50x64.pickle',
 'trainembeddings.pickle',
 'bert1000trainembeddings.pickle',
 'bert1000entitytrainembeddings.pickle',
 'uspoliticsdata',
 'testembeddings.pickle',
 'testlabels.pickle',
 'berttestembeddings.pickle',
 'berttestentityembeddings.pickle',
 'bertentitytrainembeddings.pickle',
 'berttrainembeddings.pickle',
 'trainlabels.pickle',
 'finaltrainembed.pickle',
 'ViT-B16trainembed.pickle',
 'ViT-L14trainembed.pickle',
 'RN50trainembed.pickle',
 'RN101trainembed.pickle',
 'RN50x4trainembed.pickle',
 'RN50x16trainembed.pickle',
 'ViT-B16testembeddings.pickle',
 'ViT-L14testembeddings.pickle',
 'RN50testembeddings.pickle',
 'RN101testembeddings.pickle',
 'RN50x4testembeddings.pickle',
 'RN50x16testembeddings.pickle',
 'uspoliticsfinaltrainembed.pi

In [ ]:
import json
with open('constraint22_dataset_unseen_test_gold_labels.jsonl') as f:
    testdata = [json.loads(line) for line in f]
print(testdata[4])

{'OCR': 'TOGETHER THEY RAPED A 13 YR OLD GIRL\nTHEY ARE BACKIN COURT DECEMBER 16/2016\n', 'image': 'memes_1731.png', 'hero': [], 'villain': ['jeffrey epstein', 'donald trump'], 'victim': ['13 year old girl'], 'other': []}


In [ ]:
print(len(testdata))

718


In [ ]:
test = []
k=0
for i in testdata :
  i['entities']=i['hero']+i['villain']+i['victim']+i['other']

  if i['image'].startswith('m'):
    for en in i['entities']:
      new_dict={}

      new_dict['image'] = i['image']
      new_dict['OCR'] = i['OCR']
      new_dict['entity'] = en
      if en in i['hero']:
        new_dict['label'] =0 #'hero'
      elif en in i['villain'] :
        new_dict['label'] =1 #'villain'
      elif en in i['victim'] :
        new_dict['label'] =2 #'victim'
      else:
        new_dict['label'] =3 #'other'


      test.append(new_dict)
  k=k+1

In [ ]:
print(test[5])
print(k)

{'image': 'memes_3330.png', 'OCR': 'APPARENTLY\nCAKES\nARE MORE IMPORTANT\nTOYOU THANALLOFTHIS:\nTAIX\nSTOPPING BAANE STRIKIS INUING THE FOICE STATE ELMINATING TLES\nBRING\nTHE\nTROOPS\nHOME\nSudget\nNOW\nLEGALIZE\nINDING THE\nBINGING OR\nTROOPS OME\nALANCIN\nTHE UDG\nLERT RIGHTS\nEND\nTHESE\nWARS\nDIT TE D\nSIPAnON Or\nCHURCHA STATE\nNO MORE\nCON CONTROL\nWARS\n', 'entity': 'people', 'label': 3}
718


In [ ]:
testimages = []
testocr = []
testentity=[]
testlabels = []


temp=''
for i in test:
  if i['image'] not in os.listdir('test_data/images'):
        continue
  testocr.append(i['OCR'])
  testentity.append(i['entity'])
  testlabels.append(i['label'])



  if i['image']!=temp:
      image = Image.open(os.path.join('test_data/images/', i['image'])).convert("RGB")
      testimages.append(preprocess(image))
      temp=i['image']
  else:
      testimages.append(testimages[-1])






  #testocr.append(i['OCR'])
  # testentity.append(i['entity'])
  # testlabels.append(i['label'])

# n=0

# for i in data_new:
#     n+=1
#     if n==3001:
#       break

#     if i['image'] not in os.listdir('images'):
#         continue


#     if i['image']!=temp:
#       image = Image.open(os.path.join('images/', i['image'])).convert("RGB")
#       images.append(preprocess(image))
#       temp=i['image']
#     else:
#       images.append(images[-1])

In [ ]:
len(testocr)

1143

In [ ]:
len(testimages)

1143

In [ ]:
#(testentity.shape)

In [ ]:
k=0
testimage_features = []#torch.Tensor(3000,640)
for i in range(381):

  testimage_input = torch.tensor(np.stack(testimages[k:k+3])).cuda()
  k+=3
  with torch.no_grad():
    testimage_features=testimage_features + model.encode_image(testimage_input).float().tolist()

In [ ]:
#testimage_input = torch.tensor(np.stack(testimages)).cuda()
testocr_tokens = clip.tokenize([ o for o in testocr],truncate=True).cuda()
testentity_tokens = clip.tokenize([ e for e in testentity]).cuda()

In [ ]:

with torch.no_grad():
    #testimage_features= model.encode_image(testimage_input).float()
    testocr_features = model.encode_text(testocr_tokens).float()
    testentity_features = model.encode_text(testentity_tokens).float()

In [ ]:
#print(testimage_features)
print(testocr_features.shape)
print(testentity_features.shape)

torch.Size([1143, 768])
torch.Size([1143, 768])


In [ ]:
testimage_features=torch.Tensor(testimage_features).cuda()

In [ ]:
test_tensor = torch.cat((testimage_features, testocr_features, testentity_features), 1)

In [ ]:
print(len(test_tensor))
print(len(testlabels))

1143
1143


In [ ]:
# size=len(input_tensor)
# xtrain=input_tensor[:int(0.7*size),]
# xval=input_tensor[int(0.7*size):,]
# ytrain=labels[:int(0.7*size)]
# yval=labels[int(0.7*size):]

In [ ]:
print(len(testlabels))

1143


In [ ]:
import pickle
with open("uspoliticsRN50x16testembed.pickle", 'wb') as f:
    pickle.dump(test_tensor, f)

# import pickle
# with open("uspoliticslabels.pickle", 'wb') as f:
#     pickle.dump(labels, f)

# import pickle
# with open("uspoliticstestlabels.pickle", 'wb') as f:
#     pickle.dump(testlabels, f)

# **Creating final data into train val test for classification**

In [ ]:

xtrain=input_tensor
xval=test_tensor
ytrain=labels
yval=testlabels


# xtrain=entity_features
# xval=testentity_features
# ytrain=labels
# yval=testlabels

In [ ]:

print(xtrain.shape)
print(xval.shape)
#xtrain=xtrain.numpy()
print(type(xtrain))
print(len(ytrain))
#print(yval.shape)

torch.Size([10280, 1536])
torch.Size([1143, 1536])
<class 'torch.Tensor'>
10280


In [ ]:
# max=0
# for i in range(len(target_entity_embeddings)):

#   if len(target_entity_embeddings[i])>max:
#     max=len(target_entity_embeddings[i])
# print(max)


# for i in range(len(target_entity_embeddings)):
#   if len(target_entity_embeddings[i]) <9216:
#     target_entity_embeddings[i] = target_entity_embeddings[i] + [0]*(9216-len(target_entity_embeddings[i]))

0

In [ ]:
# test_tensor = torch.cat((testimage_features, testocr_features, testentity_features,target_embeddings,target_entity_embeddings), 1)

# Classification Models

In [ ]:
from sklearn.metrics import f1_score

k neighbors classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=4)
xtrain2 = torch.Tensor.cpu(xtrain)


neigh.fit(xtrain2, ytrain)

KNeighborsClassifier(n_neighbors=4)

In [ ]:
xval2 = torch.Tensor.cpu(xval)

In [ ]:
ypredict = neigh.predict(xval2)

In [ ]:
(sum(ypredict==yval))/len(yval)

0.689413823272091

In [ ]:
f1_score(yval, ypredict, average='macro')

0.45554355261073987

In [ ]:
print(ypredict)
print(yval)


[1 3 3 ... 3 3 1]
[2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 0, 1, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 2, 3, 3, 3, 0, 1, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 1, 3, 3, 1, 2, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 1, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 1, 1, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

decision tree


In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(xtrain2, ytrain)

In [ ]:
ypred=clf.predict(xval2)


In [ ]:
(sum(ypred==yval))/len(yval)

0.7730528200537153

In [ ]:
f1_score(yval, ypred, average='macro')

0.3462627835132901

Stochastic gradient descent

In [ ]:
from sklearn.linear_model import SGDClassifier

cl = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
cl.fit(xtrain2, ytrain)
SGDClassifier(max_iter=5)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


SGDClassifier(max_iter=5)

In [ ]:
ypre = cl.predict(xval2)

In [ ]:
(sum(ypre==yval))/len(yval)

0.8549686660698299

In [ ]:
f1_score(yval, ypre, average='macro')

0.30040069686411147

SVM

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(xtrain2, ytrain2)

SVC()

In [ ]:
ypred = clf.predict(xval2)
(sum(ypred==yval))/len(yval)

TypeError: ignored

In [ ]:
f1_score(yval, ypred, average='macro')

0.22864955826672279

Very basic NN

In [ ]:
ytrain =torch.tensor(ytrain).cuda()
yval =torch.tensor(yval).cuda()
xtrain =torch.tensor(xtrain).cuda()
xval =torch.tensor(xval).cuda()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


input_size = 1536
hidden_size = 512
num_classes = 4
num_epochs = 16
batch_size = 32
learning_rate = 0.001

train_dataset = TensorDataset(xtrain,ytrain)

test_dataset = TensorDataset(xval,yval)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)



    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)

        # no activation and no softmax at the end
        return out

nnmodel = NeuralNet(input_size, hidden_size, num_classes).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(nnmodel.parameters(), lr=learning_rate)


n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  train_loss=0.0

  if torch.cuda.is_available():
    for data, trainlabels in train_loader:#xtrain.cuda(), ytrain.cuda()

        # Clear the gradients
        optimizer.zero_grad()
        # Forward Pass
        target = nnmodel(data)
        # Find the Loss
        loss = criterion(target,trainlabels)
        # Calculate gradients
        loss.backward()
        # Update Weights
        optimizer.step()
        # Calculate Loss
        train_loss += loss.item()

  print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / n_total_steps} \t\t Validation Loss: {train_loss / len(test_loader)}')


#nnmodel()


correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
prediction =[]
with torch.no_grad():
    for data in test_loader:
        testdata, tralabels = data
        # calculate outputs by running images through the network
        outputs = nnmodel(testdata)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        prediction.append(predicted)
        total += tralabels.size(0)
        correct += (predicted == tralabels).sum().item()

    # for i, (images, labels) in enumerate(train_loader):

    #     images = images.to(device)
    #     labels = labels.to(device)

    #     # Forward pass
    #     outputs = model(images)
    #     loss = criterion(outputs, labels)

    #     # Backward and optimize
    #     optimizer.zero_grad()
    #     loss.backward()
    #     optimizer.step()

        # if (i+1) % 100 == 0:
        #     print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


Epoch 1 		 Training Loss: 0.638728417039658 		 Validation Loss: 5.713070841299163
Epoch 2 		 Training Loss: 0.5329343032855425 		 Validation Loss: 4.766801268276241
Epoch 3 		 Training Loss: 0.45921239724255497 		 Validation Loss: 4.107399775336186
Epoch 4 		 Training Loss: 0.39861265525962253 		 Validation Loss: 3.5653687498221793
Epoch 5 		 Training Loss: 0.33410449321410673 		 Validation Loss: 2.9883790781928434
Epoch 6 		 Training Loss: 0.2787450464060588 		 Validation Loss: 2.4932195817430816
Epoch 7 		 Training Loss: 0.2346639286101975 		 Validation Loss: 2.098938472568989
Epoch 8 		 Training Loss: 0.1843194063313259 		 Validation Loss: 1.6486346899635262
Epoch 9 		 Training Loss: 0.15074950491905398 		 Validation Loss: 1.3483705717759829
Epoch 10 		 Training Loss: 0.13042282483176046 		 Validation Loss: 1.1665597109951906
Epoch 11 		 Training Loss: 0.11302801716482695 		 Validation Loss: 1.0109728201965078
Epoch 12 		 Training Loss: 0.09535657159654388 		 Validation Loss: 0.8529

In [ ]:
predictions = prediction[0]
k=0
for i in prediction:

  predictions = torch.cat((predictions,i),0)
  k=k+1

In [ ]:
print((predictions[:]))
predictions =predictions[32:]
print(len(predictions))

tensor([3, 3, 3,  ..., 3, 3, 3], device='cuda:0')
1143


In [ ]:
ypred = predictions.cpu()
yval =yval.cpu()


In [ ]:
f1_score(yval, ypred, average='macro')

0.48433538760315487

[tensor([2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0'), tensor([3, 0, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3,
        3, 1, 3, 0, 3, 3, 3, 3], device='cuda:0'), tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3,
        3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0'), tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 0, 3, 0, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0'), tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 2, 3], device='cuda:0'), tensor([3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0'), tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0'), tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3,
  

In [ ]:
print(len(prediction[-2]))

32


In [ ]:
print(40*32+10)

1290


## Calculating cosine similarity

We normalize the features and calculate the dot product of each pair.

In [ ]:
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

In [ ]:
print(similarity)

In [ ]:
count = len(descriptions)

plt.figure(figsize=(20, 14))
plt.imshow(similarity, vmin=0.1, vmax=0.3)
# plt.colorbar()
plt.yticks(range(count), texts, fontsize=18)
plt.xticks([])
for i, image in enumerate(original_images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
  plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, count - 0.5])
plt.ylim([count + 0.5, -2])

plt.title("Cosine similarity between text and image features", size=20)

# Zero-Shot Image Classification

You can classify images using the cosine similarity (times 100) as the logits to the softmax operation.

In [ ]:
from torchvision.datasets import CIFAR100

cifar100 = CIFAR100(os.path.expanduser("~/.cache"), transform=preprocess, download=True)

In [ ]:
text_descriptions = [f"This is a photo of a {label}" for label in cifar100.classes]
text_tokens = clip.tokenize(text_descriptions).cuda()

In [ ]:
with torch.no_grad():
    text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)

text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
top_probs, top_labels = text_probs.cpu().topk(5, dim=-1)

In [ ]:
plt.figure(figsize=(16, 16))

for i, image in enumerate(original_images):
    plt.subplot(4, 4, 2 * i + 1)
    plt.imshow(image)
    plt.axis("off")

    plt.subplot(4, 4, 2 * i + 2)
    y = np.arange(top_probs.shape[-1])
    plt.grid()
    plt.barh(y, top_probs[i])
    plt.gca().invert_yaxis()
    plt.gca().set_axisbelow(True)
    plt.yticks(y, [cifar100.classes[index] for index in top_labels[i].numpy()])
    plt.xlabel("probability")

plt.subplots_adjust(wspace=0.5)
plt.show()